In [7]:
import json
from gensim import corpora, models
import re
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import RegexpTokenizer


pathway = "/Users/jeremyfeagan/Library/Mobile Documents/com~apple~CloudDocs/MyGitRepo/ChatGPT Project/conversations.json"

# Load the JSON data from the file
with open(pathway, 'r', encoding='utf-8') as file:
    data = json.load(file)

In [8]:
# Ensure necessary NLTK data is downloaded
nltk.download('stopwords')

# Extend STOPWORDS with common programming and mathematical terms if necessary
extended_stopwords = STOPWORDS.union(set(['=', '+', '-', '*', '/', '(', ')', '#', '->', 'int', 'float', 'print', 'def']))

# Initialize a lemmatizer
lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jeremyfeagan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# Extract conversation texts from the 'mapping' key
text_data = []
for conversation in data:
    for key, value in conversation['mapping'].items():
        message = value.get('message')
        if message:
            content = message.get('content')
            if content:
                parts = content.get('parts')
                if parts and isinstance(parts, list):  # Ensure 'parts' is a list
                    parts_texts = []
                    for part in parts:
                        if isinstance(part, dict) and 'text' in part:
                            parts_texts.append(part.get('text', ''))
                        elif isinstance(part, str):
                            parts_texts.append(part)
                    combined_text = ' '.join(parts_texts).strip()
                    if combined_text:  # Ensure non-empty string
                        text_data.append(combined_text)

# Tokenize the text data
tokenized_text = [text.split() for text in text_data]

# Create a dictionary and document-term matrix
dictionary = corpora.Dictionary(tokenized_text)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_text]

# Train an LDA topic model
lda_model = models.LdaModel(doc_term_matrix, num_topics=5, id2word=dictionary, passes=15)

In [10]:
# Custom preprocessing function
def preprocess_text(text):
    # Remove hexadecimal codes and non-ASCII characters
    text = re.sub(r'\b[0-9a-fA-F]{4,}\b', '', text)
    # Keep only words with alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize and remove stopwords
    tokenizer = RegexpTokenizer(r'\w+')
    stop_words = set(stopwords.words('english'))
    tokens = tokenizer.tokenize(text.lower())
    filtered_tokens = [token for token in tokens if token not in stop_words and len(token) > 1]
    return filtered_tokens

# Assuming 'text_data' is a list of your conversation texts
processed_texts = [preprocess_text(text) for text in text_data]

# Create Dictionary and Corpus for LDA
dictionary = corpora.Dictionary(processed_texts)
corpus = [dictionary.doc2bow(text) for text in processed_texts]

# Apply LDA Model
lda_model = models.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=20, random_state=100)

# Display identified topics
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.027*"health" + 0.009*"may" + 0.007*"treatment" + 0.007*"valuebased" + 0.007*"data"')
(1, '0.015*"healthcare" + 0.010*"patient" + 0.010*"access" + 0.010*"reimbursement" + 0.010*"patients"')
(2, '0.043*"column" + 0.038*"nan" + 0.023*"length" + 0.022*"int" + 0.021*"dtype"')
(3, '0.016*"model" + 0.015*"word" + 0.013*"sentence" + 0.010*"return" + 0.010*"regression"')
(4, '0.008*"provide" + 0.008*"information" + 0.005*"understanding" + 0.005*"questions" + 0.005*"specific"')
(5, '0.017*"item" + 0.017*"value" + 0.012*"number" + 0.011*"landing" + 0.010*"rate"')
(6, '0.008*"agreement" + 0.007*"food" + 0.007*"heart" + 0.007*"options" + 0.007*"blood"')
(7, '0.026*"data" + 0.014*"code" + 0.012*"file" + 0.009*"import" + 0.008*"column"')
(8, '0.011*"content" + 0.011*"use" + 0.009*"users" + 0.009*"color" + 0.008*"text"')
(9, '0.007*"american" + 0.006*"like" + 0.006*"literature" + 0.005*"one" + 0.005*"time"')


In [12]:
dictionary = corpora.Dictionary(processed_texts, num_words=10000)  # Limit vocabulary to 10,000 words

TypeError: Dictionary.__init__() got an unexpected keyword argument 'num_words'

In [11]:
# Train an LDA topic model
lda_model = models.LdaModel(doc_term_matrix, num_topics=20, id2word=dictionary, passes=30)

IndexError: index 43521 is out of bounds for axis 1 with size 43521